In [1]:

import os
import sys
import subprocess
# sys.path.insert(1, f"{subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')}/logistics/tools")
TOOLS_PATH = f"{os.getenv('MASTER_PROJECT_ROOT_FOLDER')}/logistics"
sys.path.insert(0, TOOLS_PATH)
from tools import run_tools, endf_tools
import numpy as np
# import openmc
import h5py
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import seaborn as sns
# from tools import image_to_thesis
# image_to_thesis.pull_from_thesis()


In [4]:
# VERSION = "v1"
VERSION = "v3"

# REACTION = "002" # elastic scattering
# REACTION = "102" # radiative capture
# REACTION = "103" # nc 
# REACTION = "016" # 2n 
# REACTION = "052" # 2n 
# REACTION = "051" # heating 

for MT in tqdm([2, 51, 102], desc="MT"):
    res_folder = f"{VERSION}/MT{MT}"
    os.system(f"rm -rf {res_folder}")
    os.makedirs(res_folder, exist_ok=True)

    for REACTION in ["002", "051", "102"]:
        REACTION_INT = int(REACTION)

        plt.clf()
        plt.figure()

        if VERSION == "v1":
            sampled_h5_files_path = "/home/fne23_stjarnholm/nuclear_data/sandy_samples_v1/hdf5/F19"
        elif VERSION == "v2":
            sampled_h5_files_path = "/home/fne23_stjarnholm/nuclear_data/sandy_samples_v2"
        elif VERSION == "v3":
            sampled_h5_files_path = "/home/fne23_stjarnholm/nuclear_data/sandy_samples_v3"
        
        if MT is not None:
            sampled_h5_files_path = f"{sampled_h5_files_path}-MT{MT}"

        fig = plt.figure()
        ax = fig.gca()

        TEMPERATURE = 900 # K

        x_logspace = np.logspace(np.log10(1e-5), np.log10(2e7), 1000)

        df = pd.DataFrame(columns=["energy", "xs"])

        N_ITERATIONS = 20
        # for i in tqdm(range(N_ITERATIONS)):
        for i in range(N_ITERATIONS):
            h5_filename = f"{sampled_h5_files_path}/F19-{i+1}.h5"

            h5file = h5py.File(h5_filename, 'r')
            energy_group = h5file[f'F19/energy/{TEMPERATURE}K']
            reactions_group = h5file[f'F19/reactions']
            # for name, obj in sorted(list(main_group.items()))[:10]:
            #     if 'reaction_' in name:
            #         print('{}, {}'.format(name, obj.attrs['label'].decode()))

            n_elastic_xs_group = reactions_group[f'reaction_{REACTION}/{TEMPERATURE}K/xs']
            # print(list(n_elastic_group.values()))

            # Extract the cross section and energy data
            xs = n_elastic_xs_group[:]
            if len(xs) != len(energy_group[:]):
                energy = energy_group[-len(xs):]
            else:
                energy = energy_group[:]
            

            # print(energy.shape)
            # print(xs.shape)

            # Interpolate the cross section to the desired energy grid
            xs = np.interp(x_logspace, energy, xs)
            energy = x_logspace

            df = pd.concat([df, pd.DataFrame({"energy": energy, "xs": xs})], ignore_index=True)

            # F19_reconstructed = openmc.data.IncidentNeutron.from_hdf5(h5_filename)

            # Plot the sampled cross section
            # ax.loglog(energy, xs, label="Sampled")
            ax.loglog(energy, xs, label="Sampled", color="darkgrey", linewidth=0.1)

        ax.set_xlabel("Energy [eV]")
        ax.set_ylabel("Cross section [b]")
        ax.set_title(f"Sampled {endf_tools.MT_to_label(REACTION_INT, short=True)} cross section from {N_ITERATIONS} samples for F-19")

        plt.savefig(f"{res_folder}/sampled_cross_section_plot_{VERSION}_{REACTION}.png", dpi=300)
            
        plt.clf()
        plt.figure()
        ax = plt.gca()

        sns.lineplot(data=df, x="energy", y="xs", errorbar=('pi', 100), linewidth=0, label='_nolegend_')
        sns.lineplot(data=df, x="energy", y="xs", errorbar=('pi', 50), linewidth=0, label="_nolegend_")
        sns.lineplot(data=df, x="energy", y="xs", errorbar=None, linewidth=0.5, label="")
        plt.xscale("log")
        plt.yscale("log")

        ax.set_xlabel("Energy [eV]")
        ax.set_ylabel("Cross section [b]")
        ax.set_title(f"Sampled {endf_tools.MT_to_label(REACTION_INT, short=True)} cross section from {N_ITERATIONS} samples for F-19")
        # plt.legend(['Mean', '50 % percentile interval', '90 % percentile interval', '100 % percentile interval'])
        plt.legend(['100 % percentile interval', '50 % percentile interval', 'Mean'])
        plt.tight_layout()

        plt.savefig(f"{res_folder}/errorbands_sampled_cross_section_plot_{VERSION}_{REACTION}.pdf")

        plt.close('all')

MT: 100%|██████████| 3/3 [00:38<00:00, 12.85s/it]
